In [63]:
## imports
import pandas as pd
import ceseek as ce
import os
import torch
import tangermeme

gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu


device(type='cuda')

In [12]:
## load high activity library
high_activity_dev = pd.read_csv('/grid/wsbs/home_norepl/pmantill/train_EvoAug/EvoAug_Library_CEseek/high_activity_dev.csv')
high_activity_hk = pd.read_csv('/grid/wsbs/home_norepl/pmantill/train_EvoAug/EvoAug_Library_CEseek/high_activity_hk.csv')

high_activity_dev.head()

,sequence,actual_dev,pred_dev,se_dev
0,TTTGACTAAAAAATATTCAAAATATAAACCAAACCAAACAACAAGC...,8.481741,6.143590,5.466950
1,CAGTTCAACTCACTTTTTGTAATATATATATATTTGGTTAATAAAA...,8.481741,5.618230,8.199696
2,TGAATTCAATATCCAAACACTTAATCTAAACTATTTTGTTGTTTAA...,7.993373,3.993589,15.998274
3,CTGATTTCCGAGATGGCCTTATCAAGCCGCTTTATCAGCATCTGTT...,7.993373,3.941199,16.420110
4,GAACCGAAGCGAAACGTGGCACGAAGCAGCCAAGTTGCAGTGAATC...,7.901251,2.372929,30.562347


In [13]:
high_activity_dev.head()

,sequence,actual_dev,pred_dev,se_dev
0,TTTGACTAAAAAATATTCAAAATATAAACCAAACCAAACAACAAGC...,8.481741,6.143590,5.466950
1,CAGTTCAACTCACTTTTTGTAATATATATATATTTGGTTAATAAAA...,8.481741,5.618230,8.199696
2,TGAATTCAATATCCAAACACTTAATCTAAACTATTTTGTTGTTTAA...,7.993373,3.993589,15.998274
3,CTGATTTCCGAGATGGCCTTATCAAGCCGCTTTATCAGCATCTGTT...,7.993373,3.941199,16.420110
4,GAACCGAAGCGAAACGTGGCACGAAGCAGCCAAGTTGCAGTGAATC...,7.901251,2.372929,30.562347


In [135]:
# Make control sequences - for each sequence in the high activity library, make 1 10x dishuffled sequence with tangermeme
def csv_to_fasta(csv_file, fasta_file, sequence_col='sequence', id_cols=None):
    """
    Convert CSV with sequences to FASTA format
    
    Args:
        csv_file: path to input CSV
        fasta_file: path to output FASTA
        sequence_col: name of column containing sequences
        id_cols: list of columns to include in FASTA header (optional)
    """
    df = pd.read_csv(csv_file)
    
    with open(fasta_file, 'w') as f:
        for idx, row in df.iterrows():
            # Create header - use index if no id_cols specified
            if id_cols:
                header_parts = [f"{col}={row[col]}" for col in id_cols if col in df.columns]
                header = f">seq_{idx}_" + "_".join(header_parts)
            else:
                header = f">seq_{idx}"
            
            # Write FASTA entry
            f.write(f"{header}\n{row[sequence_col]}\n")
            

def make_control_sequences(df, output_file, n=10):
    from tangermeme.ersatz import dinucleotide_shuffle
    from tangermeme.utils import one_hot_encode
    from tangermeme.utils import characters
    import torch
    
    sequences = df['sequence'].tolist()
    
    with open(output_file, 'w') as f:
        for orig_idx, seq in enumerate(sequences):
            # Convert single sequence to one-hot: shape (4, seq_len)
            one_hot = one_hot_encode(seq)
            
            # Add batch dimension for dinucleotide_shuffle: shape (1, 4, seq_len)
            one_hot_batch = one_hot.unsqueeze(0)
            
            # Generate n shuffled versions: returns shape (1, 4*n, seq_len)
            shuffled_batch = dinucleotide_shuffle(one_hot_batch, n=n, random_state=None)
            
            # Remove batch dimension and reshape: (4*n, seq_len) -> (n, 4, seq_len)
            shuffled_no_batch = shuffled_batch.squeeze(0)  # (4*n, seq_len)
            shuffled_reshaped = shuffled_no_batch.view(n, 4, -1)  # (n, 4, seq_len)
            
            # Convert each shuffled sequence to string
            for shuffle_num in range(n):
                # Extract single shuffled sequence: shape (4, seq_len)
                shuffled_seq_tensor = shuffled_reshaped[shuffle_num]
                
                # Convert to string using characters function
                shuffled_seq_str = characters(shuffled_seq_tensor)
                
                # Write to file
                f.write(f">shuffle_{orig_idx}_{shuffle_num}\n{shuffled_seq_str}\n")

In [138]:
# Get target and control sequences in fasta format

if not os.path.exists('high_activity_dev_with_scores.fasta'):
    # Include metadata in headers
    csv_to_fasta('high_activity_dev.csv', 'high_activity_dev_with_scores.fasta', 
        id_cols=['actual_dev', 'pred_dev', 'se_dev'])
        
if not os.path.exists('high_activity_hk_with_scores.fasta'):
    csv_to_fasta('high_activity_hk.csv', 'high_activity_hk_with_scores.fasta', 
        id_cols=['actual_hk', 'pred_hk', 'se_hk'])

if not os.path.exists('high_activity_dev_control.fasta'):
    make_control_sequences(high_activity_dev, output_file='high_activity_dev_control.fasta', n=10)

if not os.path.exists('high_activity_hk_control.fasta'):
    make_control_sequences(high_activity_hk, output_file='high_activity_hk_control.fasta', n=10)


#load fasta and print head to confirm
fasta_file_dev = 'high_activity_dev_with_scores.fasta'
fasta_file_hk = 'high_activity_hk_with_scores.fasta'
fasta_file_dev_control = 'high_activity_dev_control.fasta'
fasta_file_hk_control = 'high_activity_hk_control.fasta'

seq_count_dev = sum(1 for line in open(fasta_file_dev) if line.startswith('>'))
seq_count_hk = sum(1 for line in open(fasta_file_hk) if line.startswith('>'))

print(f"Number of sequences in {fasta_file_dev}: {seq_count_dev}, {high_activity_dev.shape[0]}")
print(f"Number of sequences in {fasta_file_hk}: {seq_count_hk}, {high_activity_hk.shape[0]}")

print(f'Control sequences length: {len(open(fasta_file_dev_control).readlines()) / 2}')
print(f'Control sequences length: {len(open(fasta_file_hk_control).readlines()) / 2}')

with open(fasta_file_dev_control, 'r') as f:
    print("")
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())
with open(fasta_file_hk_control, 'r') as f:
    print("")
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())



Number of sequences in high_activity_dev_with_scores.fasta: 5340, 5340
Number of sequences in high_activity_hk_with_scores.fasta: 3802, 3802
Control sequences length: 53400.0
Control sequences length: 38020.0

>shuffle_0_0

TATTTTTTAAGTTTATAACGTAAAATTTACGTGCAACGTTAACATCGTTTAATTTAATTCAAAGAAATGATGAATTACTTGAGCAATGGGCTTCCAACACTTGGAAAACATAGCCTTTTGTAACATAATACTTCTGTGACTTGACAAATTGCTATGATAAATTCGCATAGTGTTTAGTCGCCATTCTTTGCTTTGGACACTACATAAAGTTCCATTAAACAAGTACTCAAGATATAAGAGCAGAACCTG

>shuffle_0_1

TCAGACGATTCGTACAAGGTTAATATTAATTTTGTTGCTGTAGGACAAATAGTTTTGTCACAAGAATAGCTTACGTCTAAGATAACTCCTTCACAAATTTTTTGCAGCGCAAATAGCATAGTTTCAACCTTGCTAATATGAACAAACTTCTTGACAATGACACCGCATGTATATAAGTTTGATTTCTAAGAATGAAAACCTTGGTTATGGTAAAATTATATTGCTTTAAACAATTACCAAACGACTTTG


>shuffle_0_0

TGTGCGTCGACATTTATATAAAATTTGGGAGACGACATGATTATTAATAAACATTCGTAATTAGTCAGTTATCGGCGGGCGCTTTCTCCAATTGATGATAATTTCACGAAATTTTACTAGAATTGTTAAACAAAATAAAACGGATATCTTGTGTCTCTTTATTGATGATTTTCATGGGGAGAAACTAACATCGTTATAAAATTCGCATCCGGGTAGCATGGATGTTGCGAGTCGCATCAGATAAA

In [139]:
## Load CEseek motifs.pwm
pwm_file = 'DeepSTARR_dm6_motifs.pwm'

obj_dev = ce.CEseek(pwm_file)
obj_hk = ce.CEseek(pwm_file)

obj_dev.dict_motif.keys(), obj_hk.dict_motif.keys()

(odict_keys(['Jra', 'GATAe', 'CTCF', 'Atf6', 'CrebA', 'Max', 'twi', 'Dref']),
 odict_keys(['Jra', 'GATAe', 'CTCF', 'Atf6', 'CrebA', 'Max', 'twi', 'Dref']))

In [143]:
# load fasta into CEseek

obj_dev.load_sequences(fasta_file_dev, fasta_input=True, out_seq_len=249, sequence_set='target')
obj_hk.load_sequences(fasta_file_hk, fasta_input=True, out_seq_len=249, sequence_set='target')
obj_dev.load_sequences(fasta_file_dev_control, fasta_input=True, out_seq_len=249, sequence_set='control')
obj_hk.load_sequences(fasta_file_hk_control, fasta_input=True, out_seq_len=249, sequence_set='control')

obj_dev.scan_motifs(num_threads=8)
obj_hk.scan_motifs(num_threads=8)

len(obj_dev.sequences_control)/len(obj_dev.sequences), len(obj_hk.sequences_control)/len(obj_hk.sequences)



(10.0, 10.0)

In [ ]:
## Get tfmodisco working so we can confirm motifs
## Need tangermemes attribution methods on my student model
## Plot Control counts vs Target counts

